### Setup and Load dataset



In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown --id 15hmSSQBP0hPCJHrQRBjyfqd_bPU29Rwr

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=15hmSSQBP0hPCJHrQRBjyfqd_bPU29Rwr
To: /content/MIND.zip
100% 44.4M/44.4M [00:00<00:00, 217MB/s]


In [ ]:
!unzip -o "MIND.zip"  -d  "/content"

Archive:  MIND.zip
  inflating: /content/MIND/behaviors.tsv  
  inflating: /content/MIND/news.tsv  


## Data Loading

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [ ]:
file="MIND/news.tsv"

###### Run below cell

In [ ]:
content_df  = pd.read_table( 'MIND/news.tsv',
            names=['newid', 'vertical', 'subvertical', 'title',
                            'abstract', 'url', 'entities in title', 'entities in abstract'],
                     usecols = ['newid','vertical', 'subvertical', 'title', 'abstract'])

In [ ]:
content_df=content_df[:10000]

## Details about dataset

In [ ]:
itemid="newid"

In [ ]:
features=['vertical',	'subvertical','title']

# Content based filtering

In [ ]:
content_df['NewTag']=""
for i in features:
  content_df['NewTag']+=(" "+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

## TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
cv =TfidfVectorizer(analyzer='word', stop_words='english')
vector = cv.fit_transform(content_df['NewTag'].values.astype('str') ).toarray()
similarity = cosine_similarity(vector)

In [ ]:
def Recommend(itemkey,contant_df,similarity):
    index  = contant_df[contant_df[itemid] == itemkey].index [0]
    #print(index)
    distance = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    result=[]
  
    for i in distance[1:11]:
      
        result.append(list(contant_df.iloc[i[0]])[0])
    return result    

## Recommendation

In [ ]:
content_df[content_df.newid=='N55528']

,newid,vertical,subvertical,title,abstract,NewTag
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",lifestyle lifestyleroyals The Brands Queen El...


In [ ]:
resuls=Recommend('N55528',content_df,similarity)
resuls

['N28614',
 'N42777',
 'N64143',
 'N39520',
 'N51340',
 'N42227',
 'N7716',
 'N24075',
 'N5672',
 'N8071']

In [ ]:
content_df[content_df.newid.isin(resuls)]

,newid,vertical,subvertical,title,abstract,NewTag
42,N8071,lifestyle,lifestyleroyals,25 Photos of the Royal Family at Balmoral Cast...,The royal family has been visiting the Scottis...,lifestyle lifestyleroyals 25 Photos of the Ro...
46,N42777,lifestyle,lifestyleroyals,Prince George's Royal Life in Photos,Photos of the future king of England who is ...,lifestyle lifestyleroyals Prince George's Roy...
107,N51340,lifestyle,lifestyleroyals,Queen Elizabeth's Cousin Says Royal Family 'Do...,Queen Elizabeth's Cousin: Royal Family 'Don't ...,lifestyle lifestyleroyals Queen Elizabeth's C...
392,N42227,lifestyle,lifestyleroyals,Queen Elizabeth's Favorite Beauty Products Hav...,"Here, all the brands the British monarch swear...",lifestyle lifestyleroyals Queen Elizabeth's F...
397,N7716,lifestyle,lifestyleroyals,What Do Prince George & Princess Charlotte Kno...,Do Prince William and Kate Middleton's kids kn...,lifestyle lifestyleroyals What Do Prince Geor...
1667,N28614,lifestyle,lifestyleroyals,A Look Back at Prince Charles and Camilla's Re...,In honor of the third season of The Crown.,lifestyle lifestyleroyals A Look Back at Prin...
3028,N64143,lifestyle,lifestyleroyals,Adorable Moments Between Queen Elizabeth and H...,These photos capture Her Majesty in her role a...,lifestyle lifestyleroyals Adorable Moments Be...
5887,N39520,lifestyle,lifestyleroyals,Queen Elizabeth Has Someone on Staff Break in ...,Queen Elizabeth Has Her Shoes Broken in Before...,lifestyle lifestyleroyals Queen Elizabeth Has...
6980,N5672,lifestyle,lifestyleroyals,Why Prince William Is Making Headlines for His...,Prince William Wears Traditional Sherwani in P...,lifestyle lifestyleroyals Why Prince William ...
7004,N24075,lifestyle,lifestylevideo,This is What Will Happen When Queen Elizabeth ...,Queen Elizabeth is the longest ruling monarch ...,lifestyle lifestylevideo This is What Will Ha...
